In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 2 : Get started with TabNet builtin algorithm for training tabular models

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_tabnet.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_tabnet.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_with_tabnet.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This tutorial demonstrates how to use the TabNet builtin algorithm service on the Vertex AI platform to train custom tabular models.

TabNet combines the best of two worlds: it is explainable (similar to simpler tree-based models) while benefiting from high performance (similar to deep neural networks). This makes it great for retailers, finance and insurance industry applications such as predicting credit scores, fraud detection and forecasting. 

TabNet uses a machine learning technique called sequential attention to select which model features to reason from at each step in the model. This mechanism makes it possible to explain how the model arrives at its predictions and helps it learn more accurate models. TabNet not only outperforms other neural networks and decision trees but also provides interpretable feature attributions. 

Research paper: [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/pdf/1908.07442.pdf)

### Objective

In this notebook, you learn how to run `Vertex AI TabNet` built algorithm for training custom tabular models.

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI TabNet`
- `Vertex AI Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

The steps performed include:

- Get the training data.
- Configure training parameters for the `Vertex AI TabNet` container.
- Train the model using `Vertex AI Training` using CSV data.
- Upload the model as a `Vertex AI Model` resource.
- Deploy the `Vertex AI Model` resource to a `Vertex AI Endpoint` resource.
- Make a prediction with the deployed model.
- Hyperparameter tuning the `Vertex AI TabNet` model.
- Train the model using `Vertex AI Training` using BigQuery table.

### Dataset

This tutorial uses the `petfinder` in the public Cloud Storage bucket `gs://cloud-samples-data/ai-platform-unified/datasets/tabular/`, which was generated from the [PetFinder.my Adoption Prediction](https://www.kaggle.com/c/petfinder-adoption-prediction). This dataset predicts how quickly an animal is adopted.


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages to execute this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade tensorflow
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform tensorboard-plugin-profile
! gcloud components update --quiet

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Import libraries and define constants

In [ ]:
import os
from datetime import datetime

import google.cloud.aiplatform as aip

%load_ext tensorboard

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training and prediction.

Set the variables `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more about [hardware accelerator support for your region](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators).

*Note*: TF releases before 2.3 for GPU support will fail to load the custom model in this tutorial. It is a known issue and fixed in TF 2.3. This is caused by static graph ops that are generated in the serving function. If you encounter this issue on your own custom models, use a container image for TF 2.3 with GPU support.

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

DEPLOY_GPU, DEPLOY_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### Set machine types

Next, set the machine types to use for training and prediction.

- Set the variables `TRAIN_COMPUTE` and `DEPLOY_COMPUTE` to configure your compute resources for training and prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

#### Set the training container

Next, you use the prebuilt `Vertex AI TabNet` container for training the model.

In [ ]:
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/tab_net_v2"

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### Set pre-built container for deployment

Set the pre-built Docker container image for prediction.


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

## Get the training data

First, you get a copy of the training data -- as a CSV file -- from a public Cloud Storage bucket and copy the training data to your Cloud Storage bucket.

In [ ]:
# Please note that if you use csv input, the first column is the label column.

IMPORT_FILE = "petfinder-tabular-classification-tabnet-with-header.csv"
TRAINING_DATA_PATH = f"{BUCKET_URI}/data/petfinder/train.csv"

! gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {TRAINING_DATA_PATH}

### Create and run `Vertex AI TabNet` training job


To train a TabNet custom model, you perform two steps: 1) create a custom training job, and 2) run the job.

#### Create custom training job

A custom training job is created with the `CustomTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The training container image.
- `model_serving_container_image_uri`: The URI of a container that can serve predictions for your model — either a prebuilt 

In [ ]:
DATASET_NAME = "petfinder"  # Change to your dataset name.

job = aip.CustomContainerTrainingJob(
    display_name=f"{DATASET_NAME}_{TIMESTAMP}",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

print(job)

## Configure parameter settings for TabNet training

The following table shows the parameters for the TabNet training job:

| Parameter | Data type | Description | Required |
|--|--|--|--|
| `preprocess` | boolean argument | Specify this to enable automatic preprocessing. | No|
| `job_dir` | string | Cloud Storage directory where the model output files will be stored. | Yes |
| `input_metadata_path` | string | The GCS path to the TabNet-specific metadata for the training dataset. Please see above on how to create the metadata. | No . |
| `training_data_path` | string | Cloud Storage pattern where training data is stored. | Yes |
| `validation_data_path` | string | Cloud Storage pattern where eval data is stored. | No |
| `test_data_path` | string | Cloud Storage pattern where test data is stored. | Yes |
| `input_type` | string | “bigquery“ or “csv“ - type of the input tabular data. If csv is mentioned then the first column is treated as target. If CSV files have a header, also pass the flag “data_has_header”. If “bigquery” is used, one can either supply training/validation data paths, or supply BigQuery project, dataset, and table names for preprocessing to produce the training and validation datasets.. | Yes |
| `model_type` | string | The learning task such as classification or regression. | Yes |
| `split_column` | string | The column name used to create the training, validation, and test splits. Values of the columns (a.k.a table['split_column']) should be either “TRAIN”, “VALIDATE”, or “TEST”. “TEST” is optional. Applicable to bigquery input only. | No. |
| `train_batch_size` | int | Batch size for training. | No - Default is 1024. |
| `eval_split` | float | Split fraction to use for the evaluation dataset, if `validation_data_path` is not provided. | No - Default is 0.2 |
| `learning_rate` | float | Learning rate for training. | No - Default is the default learning rate of the specified optimizer. |
| `eval_frequency_secs` | int | Frequency at which evaluation and checkpointing will take place.The default is 600. | No . |
| `num_parallel_reads` | int | Number of threads used to read input files. We suggest setting it equal or slightly less than the number of CPUs of the machine for maximal performance in most cases. For example, 6 per GPU is a good default choice. | Yes . |
| `optimizer` | string | Training optimizer. Lowercase string name of any TF2.3 Keras optimizer is supported ('sgd', 'adam', 'ftrl', etc.). See [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers). | No - Default is 'adam'. |
| `data_cache` | string | Choose to cache data to “memory”, “disk” or “no_cache”. For large datasets, caching the data into memory would throw out-of-memory errors, therefore, we suggest choosing “disk”. You can specify the disk size in the config file (as exemplified below). Make sure to request a sufficiently large (e.g. TB size) disk to write the data, for large (B-scale) datasets. | No, The default one is “memory” . |
| `bq_project` | string | The name of the BigQuery project. If input_type=bigquery and using the flag –preprocessing, this is required. This is an alternative to specifying train, validation, and test data paths. | No . |
| `dataset_name` | string | The name of the BigQuery dataset. If input_type=bigquery and using the flag –preprocessing, this is required. This is an alternative to specifying train, validation, and test data paths. | No . |
| `table_name` | string | The name of the BigQuery table. If input_type=bigquery and using the flag –preprocessing, this is required. This is an alternative to specifying train, validation, and test data paths. | No . |
| `loss_function_type` | string | There are several loss function types in TabNet. For regression: mse/mae are included. For classification: cross_entropy/weighted_cross_entropy/focal_loss are included | No . If the value is "default", we use mse for regression and cross_entropy for classification.|
| `deterministic_data` | boolean argument | Determinism of data reading from tabular data. The default is set to False. When setting to True, the experiment is deterministic. For fast training on large datasets, we suggest deterministic_data=False setting, albeit having randomness in the results (which becomes negligible at large datasets). Note that determinism is still not guaranteed with distributed training, as map-reduce causes randomness due to the ordering of algebraic operations with finite precision. However, this is negligible in practice, especially on large datasets. For the cases where 100% determinism is desired, besides deterministic_data=True setting, we suggest training with a single GPU (e.g. with MACHINE_TYPE="n1-highmem-8").| No, The default is False . |
| `stream_inputs` | boolean argument | Stream input data from GCS instead of downloading it locally - this option is suggested for fast runtime. | No . |
| `large_category_dim` | int | Dimensionality of the embedding - if the number of distinct categories for a categorical column is greater than large_category_thresh we use a large_category_dim dimensional embedding, instead of 1-D embedding. The default is 1. We suggest increasing it (e.g. to ~5 in most cases and even ~10 if the number of categories is typically very large in the dataset), if pushing the accuracy is the main goal, rather than computational efficiency and explainability.  | No . |
| `large_category_thresh` | int | Threshold for categorical column cardinality - if the number of distinct categories for a categorical column is greater than large_category_thresh we use a large_category_dim dimensional embedding, instead of 1-D embedding. The default is 300. We suggest decreasing it (e.g. to ~10), if pushing the accuracy is the main goal, rather than computational efficiency and explainability. | No . |
| `yeo_johnson_transform` | boolean argument | Enables trainable Yeo-Johnson Power Transform (the default is disabled). Please see this link: https://www.stat.umn.edu/arc/yjpower.pdf for more information on Yeo-Johnson Power Transform. With our implementation, the transform parameters are learnable along with TabNet, trained in an end-to-end way. | No . |
| `apply_log_transform` | boolean argument | If log transform statistics are contained in the metadata and this flag is true then the input features will be log transformed. Use false to not use the transform and true (the default) to use it. Especially for datasets with skewed numerical distributions, log transformations could be very helpful.  | No . |
| `apply_quantile_transform` | boolean argument | If quantile statistics are contained in the metadata and this flag is true then the input features will be quantile transformed. Use false to not use the transform and true (the default) to use it. Especially for datasets with skewed numerical distributions, quantile transformations could be very helpful. Currently supported for BigQuery input_type. | No . |
| `replace_transformed_features` | boolean argument | If true then if a transformation is applied to a feature that feature will be replaced. If false (the default option) then the transformed feature 'will be added as a new feature to the list of feature columns. Use true to replace the feature with the transform and false to append the transformed feature as a new column. | No . |
| `target_column` | string | name of the label column. Note that for classification, the label needs to be of String or integer type. | No . |
| `prediction_raw_inputs` | boolean argument | If we set this argument, the model serving allows us to pass the features as a dictionary of tensors instead of CSV row. | No . |
| `exclude_key` | boolean argument | If we set this argument, we exclude a key in the input/output. The key is helpful in batch prediction processes input and saves output in an unpredictable order. The key helps match the output with input. | No . |

Learn more about [Get started with builtin TabNet algorithm](https://cloud.google.com/ai-platform/training/docs/algorithms/tab-net-start)

In [ ]:
ALGORITHM = "tabnet"
MODEL_TYPE = "classification"
MODEL_NAME = f"{DATASET_NAME}_{ALGORITHM}_{MODEL_TYPE}"

OUTPUT_DIR = f"{BUCKET_URI}/{MODEL_NAME}_{TIMESTAMP}"
print("Output dir: ", OUTPUT_DIR)

CMDARGS = [
    "--preprocess",
    "--data_has_header",
    f"--training_data_path={TRAINING_DATA_PATH}",
    f"--job-dir={OUTPUT_DIR}",
    f"--model_type={MODEL_TYPE}",
    "--max_steps=2000",
    "--batch_size=4096",
    "--learning_rate=0.01",
    "--prediction_raw_inputs",
    "--exclude_key",
]

### Train the TabNet model

Use the `run` method to start training, which takes the following parameters:

- `args`: The command line arguments to be passed to the TabNet training container.
- `replica_count`: The number of worker replicas.
- `model_display_name`: The display name of the `Model` if the script produces a managed `Model`.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.

The `run` method creates a training pipeline that trains and creates a `Model` object. After the training pipeline completes, the `run` method returns the `Model` object.

In [ ]:
MODEL_DIR = OUTPUT_DIR

if TRAIN_GPU:
    model = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        sync=True,
    )
else:
    model = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

print(model.gca_resource)

#### Delete the training job

Use the `delete()` method to delete the training job.

In [ ]:
job.delete()

### Deploy the model

Before you use your model to make predictions, you need to deploy it to an `Endpoint`. You can do this by calling the `deploy` function on the `Model` resource. This will do two things:

1. Create an `Endpoint` resource for deploying the `Model` resource to.
2. Deploy the `Model` resource to the `Endpoint` resource.


The function takes the following parameters:

- `deployed_model_display_name`: A human readable name for the deployed model.
- `traffic_split`: Percent of traffic at the endpoint that goes to this model, which is specified as a dictionary of one or more key/value pairs.
   - If only one model, then specify as **{ "0": 100 }**, where "0" refers to this model being uploaded and 100 means 100% of the traffic.
   - If there are existing models on the endpoint, for which the traffic will be split, then use `model_id` to specify as **{ "0": percent, model_id: percent, ... }**, where `model_id` is the model id of an existing model to the deployed endpoint. The percents must add up to 100.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `starting_replica_count`: The number of compute instances to initially provision.
- `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.

### Traffic split

The `traffic_split` parameter is specified as a Python dictionary. You can deploy more than one instance of your model to an endpoint, and then set the percentage of traffic that goes to each instance.

You can use a traffic split to introduce a new model gradually into production. For example, if you had one existing model in production with 100% of the traffic, you could deploy a new model to the same endpoint, direct 10% of traffic to it, and reduce the original model's traffic to 90%. This allows you to monitor the new model's performance while minimizing the distruption to the majority of users.

### Compute instance scaling

You can specify a single instance (or node) to serve your online prediction requests. This tutorial uses a single node, so the variables `MIN_NODES` and `MAX_NODES` are both set to `1`.

If you want to use multiple nodes to serve your online prediction requests, set `MAX_NODES` to the maximum number of nodes you want to use. Vertex AI autoscales the number of nodes used to serve your predictions, up to the maximum number you set. Refer to the [pricing page](https://cloud.google.com/vertex-ai/pricing#prediction-prices) to understand the costs of autoscaling with multiple nodes.

### Endpoint

The method will block until the model is deployed and eventually return an `Endpoint` object. If this is the first time a model is deployed to the endpoint, it may take a few additional minutes to complete provisioning of resources.

In [ ]:
DEPLOYED_NAME = "{DATASET_NAME}_" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_COMPUTE.name,
        accelerator_count=0,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

### Get the serving signature

Next, download the model locally and query the model for its serving signature. The serving signature will be of the form:

    ( "feature_name_1",  "feature_name_2", ... )

In [ ]:
import tensorflow as tf

loaded = tf.saved_model.load(MODEL_DIR + "/model")
loaded.signatures

### Make a prediction

Finally, you make a prediction using the `predict()` method. Each instance is specified in the following dictionary format:

    { "feature_name_1": value, "feature_name_2", value, ... }

In [ ]:
prediction = endpoint.predict(
    [
        {
            "Age": 3,
            "Breed1": "Tabby",
            "Color1": "Black",
            "Color2": "White",
            "Fee": 100,
            "FurLength": "Short",
            "Gender": "Male",
            "Health": "Healthy",
            "MaturitySize": "Small",
            "PhotoAmt": 2,
            "Sterilized": "No",
            "Type": "Cat",
            "Vaccinated": "No",
        }
    ]
)

print(prediction)

## Hyperparameter Tuning

After successfully training your model, deploying it, and calling it to make predictions, you may want to optimize the hyperparameters used during training to improve your model's accuracy and performance. See the Vertex AI documentation for an overview of hyperparameter tuning and how to use it in your Vertex Training jobs.

For this example, the following runs a Vertex AI hyperparameter tuning job with 4 trials that attempts to maximize the validation AUC metric. The hyperparameters it optimizes are the number of max_steps and the learning rate.

### Create trial configuration

Next, you construct a YAML file which contains the hyperparameter trial settings.

In [ ]:
config = f"""studySpec:
  metrics:
  - metricId: auc
    goal: MAXIMIZE
  parameters:
  - parameterId: max_steps
    integerValueSpec:
      minValue: 2000
      maxValue: 3000
  - parameterId: learning_rate
    doubleValueSpec:
      minValue: 0.0000001
      maxValue: 0.1
trialJobSpec:
  workerPoolSpecs:
  - machineSpec:
      machineType: {TRAIN_COMPUTE}
      acceleratorType: NVIDIA_TESLA_V100
      acceleratorCount: 1
    replicaCount: 1
    diskSpec:
      bootDiskType: pd-ssd
      bootDiskSizeGb: 100
    containerSpec:
      imageUri: {TRAIN_IMAGE}
      args:
      - --preprocess 
      - --data_has_header
      - --training_data_path={TRAINING_DATA_PATH}
      - --job-dir={OUTPUT_DIR}
      - --batch_size=1028
      - --model_type={MODEL_TYPE}
      - --prediction_raw_inputs
"""

!echo $'{config}' > ./config.yaml

### Execute the hyperparameter tuning trials

Next, you execute the hyperparameter tuning job using the command `gcloud ai hp-tuning-jobs create`.

The job will run asynchronouosly. You can poll the status of the job using `gcloud ai hp-tuning-jobs describe`.

In [ ]:
MAX_TRIAL_COUNT=4
PARALLEL_TRIAL_COUNT=2

output = ! gcloud ai hp-tuning-jobs create \
  --config=config.yaml \
  --max-trial-count={MAX_TRIAL_COUNT} \
  --parallel-trial-count={PARALLEL_TRIAL_COUNT} \
  --region=$REGION \
  --display-name={DATASET_NAME}_{TIMESTAMP}

print(output)

DESCRIBE = output[5]
print("Describe cmd:", DESCRIBE)

#### Cancel the hyperparameter tuning job

Next, use the command `gcloud ai hp-tuning-jobs cancel` to cancel the hyperparameter tuning job.

In [ ]:
DESCRIBE = output[5]
print("Describe cmd:", DESCRIBE)

args = DESCRIBE.split(" ")
JOB_ID = args[7]

! gcloud ai hp-tuning-jobs cancel {JOB_ID} --region={REGION}

## Create and run a `Vertex AI TabNet` training job with BigQuery input

You can train a `Vertex AI TabNet` tabular model using data from either CSV input on a Cloud Storage location, or a BigQuery table. In this next example, you train a model using data from BigQuery.

### Create the BigQuery table for training data.

First, create the BQ dataset using the above csv file.

In [ ]:
# Create the BQ dataset
! bq --location={REGION} mk --dataset {PROJECT_ID}:{DATASET_NAME}
# Create the BQ table and populate it with data from the CSV file
! bq --location={REGION} load --source_format=CSV --autodetect {PROJECT_ID}:{DATASET_NAME}.train {BUCKET_URI}/data/petfinder/train.csv

#### Create custom training job

A custom training job is created with the `CustomTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The training container image.
- `model_serving_container_image_uri`: The URI of a container that can serve predictions for your model — either a prebuilt 

In [ ]:
job = aip.CustomContainerTrainingJob(
    display_name=f"{DATASET_NAME}_{TIMESTAMP}",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

print(job)

### Configure parameter settings for TabNet training

Next, configure the parameter settings for training with BigQuery input.

In [ ]:
BQ_PROJECT = f"{PROJECT_ID}"
BQ_TABLE = "train"
TARGET_COLUMN = "Adopted"

CMDARGS = [
    "--eval_frequency_secs=10800",
    "--input_type=bigquery",
    "--preprocess",
    f"--model_type={MODEL_TYPE}",
    "--stream_inputs",
    "--max_steps=3000",
    f"--bq_project={BQ_PROJECT}",
    f"--dataset_name={DATASET_NAME}",
    f"--table_name={BQ_TABLE}",
    f"--target_column={TARGET_COLUMN}",
    "--num_parallel_reads=2",
    "--optimizer_type=adam",
    "--data_cache=disk",
    "--deterministic_data=False",
    "--loss_function_type=weighted_cross_entropy",
    "--replace_transformed_features=True",
    "--apply_quantile_transform=True",
    "--apply_log_transform=True",
    "--batch_size=32768",
    "--learning_rate=0.01",
    "--prediction_raw_inputs",
    "--exclude_key",
    f"--job-dir={OUTPUT_DIR}",
]

### Train the TabNet model

Use the `run` method to start training, which takes the following parameters:

- `args`: The command line arguments to be passed to the TabNet training container.
- `replica_count`: The number of worker replicas.
- `model_display_name`: The display name of the `Model` if the script produces a managed `Model`.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.

The `run` method creates a training pipeline that trains and creates a `Model` object. After the training pipeline completes, the `run` method returns the `Model` object.

In [ ]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
print("Service Account:", SERVICE_ACCOUNT)


MODEL_DIR = OUTPUT_DIR

if TRAIN_GPU:
    model_bq = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )
else:
    model_bq = job.run(
        model_display_name=f"{DATASET_NAME}_{TIMESTAMP}",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        service_account=SERVICE_ACCOUNT,
        sync=True,
    )

print(model_bq.gca_resource)

#### Delete the training job

Use the `delete()` method to delete the training job.

In [ ]:
job.delete()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
delete_bucket = False

# Delete BQ table
! bq rm -f {PROJECT_ID}:{DATASET_NAME}.train

try:
    endpoint.undeploy_all()
    endpoint.delete()
    model.delete()
    model_bq.delete()
except Exception as e:
    print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI